In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
from torch_geometric.data import Dataset, DataLoader

In [9]:
# defining a seed for reproducible results
np.random.seed(69)

In [10]:
# Check if CUDA is available, then MPS, otherwise use CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.empty_cache()
    # cluster path
    raster_path = "../scratch/vector"
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    # local path
    raster_path = "../data.nosync/vector"
else:
    device = torch.device("cpu")
    # local path
    raster_path = "../data.nosync/vector"

print(f"Device set to: {device}")

Device set to: mps


In [11]:
# operators are always specified in this order
operator_order = ("elimination", "aggregation", "typification", "displacement", "enlargement", "simplification")

In [12]:
# Define DIN font for plots if working locally
if not torch.cuda.is_available():
    plt.rcParams["font.family"] = "DIN Alternate"

### Loading the data

In [14]:
# Setting up a Dataset object for DataLoader
class BuildingVectorDataset(Dataset):
    def __init__(self, path, transform=None):
        # store directory of individual files
        self.path = path
        # get filenames of individual files
        self.filenames = os.listdir(path)

        # store transformation
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        # get filename associated with given index
        filename = self.filenames[index]

        # load the file with the filname
        # graph = torch.load(os.path.join(self.path, filename))

### Model design

In [ ]:
# Classification head

In [ ]:
class MultiLabelGCN(nn.Module):
    def __init__(self, num_features, num_classes):
        super(MultiLabelGCN, self).__init__()
        self.conv1 = pyg_nn.GCNConv(num_features, 16)  # First GCN layer
        self.conv2 = pyg_nn.GCNConv(16, num_classes)   # Second GCN layer maps to the number of classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GCN layer with ReLU activation
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)  # Apply dropout during training

        # Second GCN layer
        x = self.conv2(x, edge_index)

        # Apply sigmoid to output layer to get probabilities for each class
        return torch.sigmoid(x)

### Elimination model

### Selection model

### Evaluation

In [ ]:
# https://www.youtube.com/watch?v=-UjytpbqX4A at 1:06:00
# --> Visualize node embeddings